# Set up

In [2]:
# import libraries 
import pandas as pd
import numpy as np
from pathlib import Path
import os
from typing import Optional


Set output directory: 

In [ ]:
# Set the working directory
project_dir = Path("/Users/Jennie/Desktop/WashU/Rotation_labs/Griffith Lab/Neoantigen ML project")
os.chdir(project_dir)

# Define output directory
outdir = Path.cwd() / "output_python" / "02_make_meta2.ipynb"
os.makedirs(outdir, exist_ok=True)

print(f"Output directory created at: {outdir}")


Output directory created at: /Users/Jennie/Desktop/WashU/Rotation_labs/Griffith Lab/Neoantigen ML project/output_python/02_make_meta2.ipynb


Read manually edited metadata from data/meta_manual.csv

In [4]:
# Define file path
file_path = Path("data/meta_manual.csv")

# Load the CSV file into a pandas DataFrame
meta_manual = pd.read_csv(file_path)

# Display the DataFrame
meta_manual.head()  # Shows the first few rows


# Filter rows where include_ML is "Yes" or external_validation is "Yes"
patient_id_ml = meta_manual.loc[
    (meta_manual["include_ML"] == "Yes") | (meta_manual["external_validation"] == "Yes"),
    "patient_id"
].tolist()

# Print or check the output
print(patient_id_ml)


['TWJF-5120-04', 'mcdb044', 'TWJF-10146-MD017-0016', 'TWJF-5120-06', 'TWJF-5120-07', 'TWJF-5120-05', 'mcdb041_original', 'mcdb041_new', 'mcdb045', 'mcdb047', 'TWJF-10146-MO011-0020', 'TWJF-5120-09', 'TWJF-5120-11', 'TWJF-10146-MO011-0024', 'mcdb046', 'TWJF-10146-MO011-0021', 'TWJF-10146-MO011-0026', 'mcdb048', 'JLF-100-042', 'TWJF-5120-13', 'TWJF-10146-MO011-0029', 'TWJF-10146-MO011-9999_TEST', 'TWJF-5120-16', 'TWJF-5120-19', 'TWJF-5120-17', 'TWJF-5120-18', '5120-39', 'CTEP-10146-MD017-0052', 'G110_Region1', 'G110_Region2', 'G113_Region1', 'TWJF-5120-41', 'TWJF-5120-42', '10146-0053', '10146-0054', '04143-004']


Get a list of patient_id that will be used in the ML study (NEW: also add samples for external validation/prediction)

# Update counts

Read the itb_review file of each patient, extract: 
* the total number of peptides for review in that patient
* the total number of "Pass" in the Tier column
* the total number of "Accept" in the Evaluation column

In [5]:
def extract_num_peptides(patient_id, project_dir):
    # Path to the peptide review files
    itb_path = project_dir / "data" / "itb_review"

    # Find files that match the patient ID
    itb_files = list(itb_path.glob(f"*{patient_id}*.tsv"))

    if not itb_files:
        print(f"[SKIP] No file found for patient {patient_id}")
        return None

    print(f"[PROCESS] Counting peptides for patient {patient_id}")

    try:
        # Attempt something that might raise an error
        # Read the first matching file
        itb_file = pd.read_csv(itb_files[0], sep="\t")

        # Compute counts
        n_peptides = len(itb_file)
        n_pass = (itb_file['Tier'] == "Pass").sum() if 'Tier' in itb_file.columns else None
        n_accept = (itb_file['Evaluation'] == "Accept").sum() if 'Evaluation' in itb_file.columns else None

        return pd.DataFrame({
            'patient_id': [patient_id],
            'n_peptides': [n_peptides],
            'n_pass': [n_pass],
            'n_accept': [n_accept]
        })

    except Exception as e: # Exception is the general type of error. as e gives you the actual error object (e), so you can print or log the message like: file not found, column missing, etc.
        print(f"[ERROR] Failed to process {patient_id}: {e}")
        return None
    
# extract_num_peptides('TWJF-5120-06', project_dir) # test the function


In [ ]:
# Apply to each patient and concatenate results
count_df_list = [extract_num_peptides(pid, project_dir) for pid in patient_id_ml]
count_df = pd.concat([df for df in count_df_list if df is not None], ignore_index=True)

# Merge back into the original metadata
meta_count = meta_manual.merge(count_df, on="patient_id", how="left")

# Display the final DataFrame
meta_count.head()


[PROCESS] Counting peptides for patient TWJF-5120-04
[PROCESS] Counting peptides for patient mcdb044
[PROCESS] Counting peptides for patient TWJF-10146-MD017-0016
[PROCESS] Counting peptides for patient TWJF-5120-06
[PROCESS] Counting peptides for patient TWJF-5120-07
[PROCESS] Counting peptides for patient TWJF-5120-05
[PROCESS] Counting peptides for patient mcdb041_original
[PROCESS] Counting peptides for patient mcdb041_new
[PROCESS] Counting peptides for patient mcdb045
[PROCESS] Counting peptides for patient mcdb047
[PROCESS] Counting peptides for patient TWJF-10146-MO011-0020
[PROCESS] Counting peptides for patient TWJF-5120-09
[PROCESS] Counting peptides for patient TWJF-5120-11
[PROCESS] Counting peptides for patient TWJF-10146-MO011-0024
[PROCESS] Counting peptides for patient mcdb046
[PROCESS] Counting peptides for patient TWJF-10146-MO011-0021
[PROCESS] Counting peptides for patient TWJF-10146-MO011-0026
[PROCESS] Counting peptides for patient mcdb048
[PROCESS] Counting pept

,patient_id,tumor_type,trial,driver_genes,itb_review_path,all_epitopes_path,class2_path,Note,include_ML,external_validation,n_peptides,n_pass,n_accept
0,mcdb043,Colon adenocarinoma,JLF,"TP53,FAT3,PRDM2",/storage1/fs1/mgriffit/Active/JLF_MCDB/cases/m...,/storage1/fs1/mgriffit/Active/JLF_MCDB/cases/m...,/storage1/fs1/mgriffit/Active/JLF_MCDB/cases/m...,itb_review has missing columns: Best Transcrip...,No,No,NaN,NaN,NaN
1,TWJF-5120-02,Pancreas,Leidos/NCI,"TP53,ACVR1B,ACVR2A,AKT2,APC,ATRX,BRAF,BRCA2,DA...",/storage1/fs1/gillandersw/Active/Project_0001_...,/storage1/fs1/gillandersw/Active/Project_0001_...,/storage1/fs1/gillandersw/Active/Project_0001_...,itb_review file and class2 file rows don't match,No,No,NaN,NaN,NaN
2,TWJF-5120-03,Pancreas,Leidos/NCI,"TP53,ACVR1B,ACVR2A,AKT2,APC,ATRX,BRAF,BRCA2,DA...",missing,NaN,NaN,"this was a screen fail, do not include this case.",No,No,NaN,NaN,NaN
3,TWJF-5120-04,Pancreas,Leidos/NCI,"TP53,ACVR1B,ACVR2A,AKT2,APC,ATRX,BRAF,BRCA2,DA...",/storage1/fs1/gillandersw/Active/Project_0001_...,/storage1/fs1/gillandersw/Active/Project_0001_...,/storage1/fs1/gillandersw/Active/Project_0001_...,has an extra column called Percentile.Fail,Yes,No,40.0,9.0,15.0
4,mcdb044,Osteosarcoma,JLF,"TP53,EXT1,EXT2,RECQL4,WRN",/storage1/fs1/mgriffit/Active/JLF_MCDB/cases/m...,/storage1/fs1/mgriffit/Active/JLF_MCDB/cases/m...,/storage1/fs1/mgriffit/Active/JLF_MCDB/cases/m...,NaN,Yes,No,38.0,2.0,12.0


# Purity 

* Purity is an estimate of the amount of tumor cells in the resected tissue
* It is calculated from tumor driver genes (clonal mutations)
  * Clonal mutations are early-mutated genes in the tumor cells, such that all cancer cells (a clone) has this mutation
  * Subclonal mutations are mutations gained in a subclone, so only a portion of the cencer cells have this mutation
* To get purity score for each patient: 
  * Find a set of driver genes of each patient's cancer type (from CancerGeneCensus file)
  * Search in the patient's itb_review file for mutation in these driver genes (`Gene` column)
  * Purity = 2*`DNA_VAF` of the driver gene
  * VAF is the percentage of sequence reads observed matching a specific DNA variant divided by the overall coverage at that locus.



In [ ]:

def extract_purity_from_driver(patient_id, project_dir, meta_count):
    # Step 1: Get driver genes for this patient
    match = meta_count[meta_count['patient_id'] == patient_id]
    if match.empty:
        print(f"[SKIP] No metadata found for patient {patient_id}")
        return None

    driver_genes_str = match.iloc[0].get('driver_genes', None)
    if pd.isna(driver_genes_str):
        print(f"[SKIP] No driver genes for patient {patient_id}")
        return None

    # Step 2: Locate the file
    itb_path = project_dir / "data" / "itb_review"
    itb_files = list(itb_path.glob(f"*{patient_id}*.tsv"))
    if not itb_files:
        print(f"[SKIP] No file found for patient {patient_id}")
        return None

    print(f"[PROCESS] Calculating purity for patient {patient_id}")
    try:
        df = pd.read_csv(itb_files[0], sep="\t")

        # Ensure columns exist
        if 'DNA VAF' not in df.columns or 'Gene' not in df.columns:
            print(f"[WARN] Missing DNA VAF or Gene column for {patient_id}")
            return None

        # Convert VAF safely
        df['DNA VAF'] = pd.to_numeric(df['DNA VAF'], errors='coerce')

        # Parse driver genes
        driver_genes = [g.strip() for g in str(driver_genes_str).split(',')]

        # Subset to only driver genes
        driver_vaf_df = df[df['Gene'].isin(driver_genes)][['ID', 'Gene', 'DNA VAF']].dropna()

        vaf_for_purity = pd.DataFrame()

        # === Begin Decision Logic ===
        if len(driver_vaf_df) == 1 and driver_vaf_df['DNA VAF'].iloc[0] < 0.5:
            vaf_for_purity = driver_vaf_df.copy()
            vaf_for_purity['purity_from_driver'] = "YES"

        elif len(driver_vaf_df) == 1 and driver_vaf_df['DNA VAF'].iloc[0] >= 0.5:
            below_05 = df[df['DNA VAF'] < 0.5].sort_values('DNA VAF', ascending=False)
            if not below_05.empty:
                highest_vaf_row = below_05[['ID', 'Gene', 'DNA VAF']].iloc[0:1].copy()
                highest_vaf_row['purity_from_driver'] = "NO"
                vaf_for_purity = highest_vaf_row

        elif len(driver_vaf_df) > 1 and any(driver_vaf_df['DNA VAF'] < 0.5):
            below_05 = driver_vaf_df[driver_vaf_df['DNA VAF'] < 0.5].sort_values('DNA VAF', ascending=False)
            if not below_05.empty:
                highest = below_05.iloc[0:1].copy()
                highest['purity_from_driver'] = "YES"
                vaf_for_purity = highest

        elif len(driver_vaf_df) > 1 and not any(driver_vaf_df['DNA VAF'] < 0.5):
            below_05 = df[df['DNA VAF'] < 0.5].sort_values('DNA VAF', ascending=False)
            if not below_05.empty:
                highest = below_05[['ID', 'Gene', 'DNA VAF']].iloc[0:1].copy()
                highest['purity_from_driver'] = "NO"
                vaf_for_purity = highest

        elif len(driver_vaf_df) == 0:
            below_05 = df[df['DNA VAF'] < 0.5].sort_values('DNA VAF', ascending=False)
            if not below_05.empty:
                highest = below_05[['ID', 'Gene', 'DNA VAF']].iloc[0:1].copy()
                highest['purity_from_driver'] = "NO"
                vaf_for_purity = highest

        # Finalize output
        if not vaf_for_purity.empty:
            vaf_for_purity['purity'] = vaf_for_purity['DNA VAF'] * 2
            vaf_for_purity['patient_id'] = patient_id
            return vaf_for_purity[['patient_id', 'ID', 'Gene', 'DNA VAF', 'purity_from_driver', 'purity']]

        else:
            print(f"[WARN] No valid purity row for patient {patient_id}")
            return None

    except Exception as e:
        print(f"[ERROR] {patient_id}: {e}")
        return None


In [ ]:
purity_df_list = [
    extract_purity_from_driver(pid, project_dir, meta_count)
    for pid in meta_count['patient_id']
]

purity_df = pd.concat([df for df in purity_df_list if df is not None], ignore_index=True)


# Merge the purity results back into the main metadata table
# Merge full purity information into the metadata
meta_full = meta_count.merge(purity_df, on='patient_id', how='left')

# Preview the merged DataFrame
meta_full.head()


[SKIP] No file found for patient mcdb043
[SKIP] No file found for patient TWJF-5120-02
[SKIP] No file found for patient TWJF-5120-03
[PROCESS] Calculating purity for patient TWJF-5120-04
[PROCESS] Calculating purity for patient mcdb044
[PROCESS] Calculating purity for patient TWJF-10146-MD017-0016
[PROCESS] Calculating purity for patient TWJF-5120-06
[PROCESS] Calculating purity for patient TWJF-5120-07
[PROCESS] Calculating purity for patient TWJF-5120-05
[PROCESS] Calculating purity for patient 7071-04
[PROCESS] Calculating purity for patient mcdb041_original
[PROCESS] Calculating purity for patient mcdb041_new
[PROCESS] Calculating purity for patient mcdb045
[PROCESS] Calculating purity for patient mcdb047
[PROCESS] Calculating purity for patient TWJF-10146-MO011-0020
[PROCESS] Calculating purity for patient TWJF-5120-09
[PROCESS] Calculating purity for patient TWJF-5120-11
[SKIP] No driver genes for patient TWJF-04143-002
[PROCESS] Calculating purity for patient TWJF-10146-MO011-00

,patient_id,tumor_type,trial,driver_genes,itb_review_path,all_epitopes_path,class2_path,Note,include_ML,external_validation,n_peptides,n_pass,n_accept,ID,Gene,DNA VAF,purity_from_driver,purity
0,mcdb043,Colon adenocarinoma,JLF,"TP53,FAT3,PRDM2",/storage1/fs1/mgriffit/Active/JLF_MCDB/cases/m...,/storage1/fs1/mgriffit/Active/JLF_MCDB/cases/m...,/storage1/fs1/mgriffit/Active/JLF_MCDB/cases/m...,itb_review has missing columns: Best Transcrip...,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TWJF-5120-02,Pancreas,Leidos/NCI,"TP53,ACVR1B,ACVR2A,AKT2,APC,ATRX,BRAF,BRCA2,DA...",/storage1/fs1/gillandersw/Active/Project_0001_...,/storage1/fs1/gillandersw/Active/Project_0001_...,/storage1/fs1/gillandersw/Active/Project_0001_...,itb_review file and class2 file rows don't match,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TWJF-5120-03,Pancreas,Leidos/NCI,"TP53,ACVR1B,ACVR2A,AKT2,APC,ATRX,BRAF,BRCA2,DA...",missing,NaN,NaN,"this was a screen fail, do not include this case.",No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TWJF-5120-04,Pancreas,Leidos/NCI,"TP53,ACVR1B,ACVR2A,AKT2,APC,ATRX,BRAF,BRCA2,DA...",/storage1/fs1/gillandersw/Active/Project_0001_...,/storage1/fs1/gillandersw/Active/Project_0001_...,/storage1/fs1/gillandersw/Active/Project_0001_...,has an extra column called Percentile.Fail,Yes,No,40.0,9.0,15.0,chr18-51065548-51065549-G-A,SMAD4,0.317,YES,0.634
4,mcdb044,Osteosarcoma,JLF,"TP53,EXT1,EXT2,RECQL4,WRN",/storage1/fs1/mgriffit/Active/JLF_MCDB/cases/m...,/storage1/fs1/mgriffit/Active/JLF_MCDB/cases/m...,/storage1/fs1/mgriffit/Active/JLF_MCDB/cases/m...,NaN,Yes,No,38.0,2.0,12.0,chr2-209694772-209694800-GGCTACTGTGTGTTCAATAAG...,MAP2,0.428,NO,0.856


# Export final metadata: 

In [9]:
# Define output file path (if not already defined)
output_path = outdir / "metadata_count_purity.xlsx"

# Save to Excel
meta_full.to_excel(output_path, index=False)

print(f"Saved to: {output_path}")

# Define output file path for CSV
output_path_csv = outdir / "metadata_count_purity.csv"

# Save to CSV
meta_full.to_csv(output_path_csv, index=False)

print(f"Saved to: {output_path_csv}")


Saved to: /Users/Jennie/Desktop/WashU/Rotation_labs/Griffith Lab/Neoantigen ML project/output_python/02_make_meta2.ipynb/metadata_count_purity.xlsx
Saved to: /Users/Jennie/Desktop/WashU/Rotation_labs/Griffith Lab/Neoantigen ML project/output_python/02_make_meta2.ipynb/metadata_count_purity.csv
